# Westeros Tutorial - Introducing soft constraints

In the baseline tutorial, we added dynamic constraints on activity via the parameter `"growth_activity_up"` for the electricity generation technologies. As a result, when we added an emission tax, `"wind_ppl"` was scaled up at the maximum rate of 10% annually in the last period.

In this tutorial, we are going to explore how to provide additional flexibility to the dynamic growth constraints. We will explore so-called [`soft constraints`](https://docs.messageix.org/en/stable/model/MESSAGE/parameter_def.html?highlight=soft%20constraint#dynamic-constraints-on-new-capacity-and-activity). Soft constraints can be configured to provide a relaxation for both activity and capacity related dynamic constraints. At a certain cost, additional annual growth rate can be realized. The cost can be absolute or defined as a share of the levelized cost (see details [here](https://docs.messageix.org/en/stable/model/MESSAGE/parameter_def.html?highlight=soft%20constraint#cost-parameters-for-soft-relaxations-of-dynamic-constraints)).

Providing this additional flexibility to dynamic constraints can be useful for assessing mitigation pathways. While without these e.g. certain emission targets may not be achievable, the additional flexibility provided reflects the fact that investments i.e. technology diffusion, can be realized faster taking into account higher costs. This can provide interesting insights as to where additional subsidies or policies can help pursue more ambitious targets.

Further information can be found in [Kepp and Strubegger, 2010](https://doi.org/10.1016/j.energy.2010.01.019). 

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros tutorial on emission taxes (`westeros_emissions_taxes.ipynb`) and solved it successfully

In [ ]:
import ixmp

mp = ixmp.Platform()

## Load existing and clone to new scenario
We load the existing scenario '*carbon_tax*' and clone it to a new scenario '*carbon_tax_soft_constraints*' to which we will add soft constraints for the upper dynamic growth constraint.

In [ ]:
import message_ix

model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="carbon_tax")
scen = base.clone(
    model, "carbon_tax_soft_constraints", "adding_soft_constraints", keep_solution=False
)
scen.check_out()

## Retrieve parameters
We will retrieve those parameters necessary to perform subsequent additions of parameters.

In [ ]:
model_horizon = base.set("year")
country = "Westeros"

## Add soft activity up for `"wind_ppl"`

Recall that when setting up the Westeros baseline scenario, we added the parameter `"growth_activity_up"` for `"wind_ppl"`, with a value of 10%. As the growth rate is an annual value and the duration of the periods in this example are 10 years, activity within a single period can increase by a maximum of 259% ((1 + .10)^10).

We will now add a soft constraint for the year 720, allowing additional growth of up to 1% per year. This means that activity can increase by a maximum of 259% + 10% (((1 + .01)^10) - 1). The costs will be defined in relative terms i.e. relative to the [levelized cost](https://docs.messageix.org/en/stable/model/MESSAGE/scaling_investment_costs.html?highlight=levelized%20cost#levelized-capital-costs). By specifying the `"level_cost_activity_soft_up"` as 1%, we are defining that per unit of activity from the previous period, used to determine the allowable additional activity, 1% of the `levelized_cost` are applied (see [link](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html#objective-function) for more information).

### Define additional annual growth.
We will allow `"wind_ppl"` to grow an additional 1% annually in the year 720.

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "node_loc": country,
        "technology": "wind_ppl",
        "year_act": model_horizon,
        "time": "year",
        "value": [0.0, 0.0, 0.0, 0.01],
        "unit": "-",
    }
)
scen.add_par("soft_activity_up", df)

### Define costs for additional growth.
As previously explained the relative costs to increase the activity of `"wind_ppl"` will be set to 1% of the `levelized_cost` of `"wind_ppl"`.

In [ ]:
df = pd.DataFrame(
    {
        "node_loc": country,
        "technology": "wind_ppl",
        "year_act": model_horizon,
        "time": "year",
        "value": ".01",
        "unit": "-",
    }
)
scen.add_par("level_cost_activity_soft_up", df)

## Commit and solve

In [ ]:
scen.commit("soft constraints added for wind_ppl")
scen.set_as_default()
scen.solve()

## Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

rep_base = Reporter.from_scenario(base)
prepare_plots(rep_base)

rep_scen = Reporter.from_scenario(scen)
prepare_plots(rep_scen)

### Activity
***
When comparing the results of the scenario '*carbon_tax*', `"coal_ppl"` still contributed to the eletricity generation mix in 710.

In [ ]:
rep_base.set_filters(t=["coal_ppl", "wind_ppl"])
rep_base.get("plot activity")

With the additional growth permitted in 720, `"coal_ppl"` is now completely phased out.

In [ ]:
rep_scen.set_filters(t=["coal_ppl", "wind_ppl"])
rep_scen.get("plot activity")

In the figure below the dark-blue bars represent the maximum activity for each period. This is calculated based on the activity of the preceding period and accounting for the annual growth of 10% (`"growth_activity_up"`). The orange bar shows that additional activity based on the soft constraint added for `"wind_ppl"` in 720.
The lines compare the results (`var("ACT")`) for `"wind_ppl"` of the carbon tax scenario without (gold  line) and with soft constraints (grey line). In the scenario with soft constraints, you can see that already in 710 `"wind_ppl"` has increased activity, so that full use can be made of the relaxation provided by the soft constraints in 720.

<img src='_static/soft-constraint.PNG' width='600'>

In [ ]:
mp.close_db()